In [1]:
from model_training import MLPipeline
from feature_engineering import FeaturePipeline
import yaml
from helpers.utils import read_csv, convert_to_date
import pandas as pd
import gc
import argparse
import datetime


def forecasting_pipeine(cfg):
    '''This function creates features, trains a model and saves a file with forecasted values'''

    data = create_data(cfg)

    FeatureBuilder = FeaturePipeline(**cfg['MODEL'])
    FeatureBuilder.create_features(data)

    MLBuilder = MLPipeline(**cfg['MODEL'])

    MLBuilder.ml_pipeline(FeatureBuilder.features)

    save_predictions(MLBuilder.predictions['merged'], cfg['OUTPUT'])


def save_predictions(predictions, path):
    df = predictions[['date', 'hierarchy1_id',
                      'storetype_id', 'sales', 'lgb_predicted']]
    df.to_csv(path, index=False)


def update_dates(cfg):
    for key, value in cfg['MODEL'].items():
        if 'date' in key:
            cfg['MODEL'][key] = convert_to_date(value)
    return cfg


def set_config(filepath):
    '''This function reads configuration file'''
    with open(filepath, 'r') as ymlfile:
        cfg = yaml.load(ymlfile)

    return update_dates(cfg)


def create_data(cfg):
    '''This function reads files and merges into one dataframe'''
    datasets = {}
    for key, value in cfg['INPUT'].items():
        datasets[key] = read_csv(value, nrows=10)

    data = pd.merge(datasets['sales'], datasets['taxonomy'], how='inner', on=['product_id'])\
        .merge(datasets['stores'], how='inner', on=['store_id'])

    del datasets
    gc.collect()
    return data


# parser = argparse.ArgumentParser(description='ml_pipeline')
# parser.add_argument('--config_path', action="store",
#                     dest='config_path', default='../prediction_config.yaml')
cfg = set_config('../prediction_config.yaml')
forecasting_pipeine(cfg)


KeyError: 'ML'

NameError: name 'cfg' is not defined

In [ ]:
cfg = set_config

Mem. usage decreased from  0.00 Mb to  0.00 Mb (6.2% reduction)
Mem. usage decreased from  0.00 Mb to  0.00 Mb (3.9% reduction)
Mem. usage decreased from  0.00 Mb to  0.00 Mb (3.4% reduction)


44